<a href="https://colab.research.google.com/github/suyeonb/AI_lab_baesuyeon/blob/main/week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchaudio

bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model()
labels = bundle.get_labels()

def speech_to_text(audio_file):
    """음성 파일을 텍스트로 변환"""
    # 오디오 파일 로드
    waveform, sr = torchaudio.load(audio_file)

    # 샘플레이트 맞추기 (16kHz)
    if sr != bundle.sample_rate:
        waveform = torchaudio.functional.resample(waveform, sr, bundle.sample_rate)

    # Mono로 변환
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # 모델 추론
    with torch.inference_mode():
        emissions, _ = model(waveform)

    # Greedy decoding
    indices = torch.argmax(emissions, dim=-1)[0]
    indices = torch.unique_consecutive(indices, dim=-1)
    indices = [i for i in indices if i != 0]  # blank 토큰 제거
    transcript = ''.join([labels[i] for i in indices])

    return transcript

# ==================== 사용 예시 ====================
if __name__ == "__main__":
    # 오디오 파일 경로
    audio_file = "week4_data.mp3"

    try:
        # STT 실행
        result = speech_to_text(audio_file)
        print(f"인식 결과: {result}")

    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {audio_file}")
    except Exception as e:
        print(f"에러 발생: {e}")

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth


100%|██████████| 360M/360M [00:00<00:00, 391MB/s]
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/au

인식 결과: LADIES|AND|GENTLEMEN|WELCOME|TO|THE|NATIONAL|MENIOL|CENTRE|THE|PERFORMANCE|WILL|BEGIN|SHORTLY|PLEASE|TURN|OFF|ANY|SELPHONES|AND|KEEP|IN|MIND|THE|RECORDING|OR|TAKING|PHOTOGRAPHS|OF|THE|PERFORMANCE|IS|NOT|PERMITTED|WITHOUT|AGREEMENT|FOR|YOUR|SAFETY|PLEASE|REFRAIN|FROM|CHANGING|SEATS|DURING|THE|SHOW|AND|LOCATE|THE|EMERGENCY|EXIT|AND|ESCAPE|ROOT|NEAREST|TO|YOU|
